In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta
import pickle
import requests
import numpy as np
from glicko2 import Glicko2
from trueskill import TrueSkill
from scipy.stats import norm
from collections import defaultdict

## First look at the data

This dataset contains results from every Bundesliga match from 1993-1994 to 2021-2022. It also includes half time results, but only since 1995-96. Columns include Division (denoted as D1), HomeTeam, AwayTeam, FTHG (final time home goals), FTAG (final time away goals), FTR (full time result), HTHG (half time home goals), HTAG (half time away goals), HTR (half time result), and season.

Data compiled into one file from this site: http://www.football-data.co.uk/germanym.php

In [34]:
with open(file) as f:
    print(f)

<_io.TextIOWrapper name='D1 (1).csv' mode='r' encoding='UTF-8'>


Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY',
       'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'GBH',
       'GBD', 'GBA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD',
       'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA',
       'BSH', 'BSD', 'BSA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD',
       'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5',
       'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA',
       'PSCH', 'PSCD', 'PSCA'],
      dtype='object')

In [36]:
encodings = ['ascii', 'big5', 'big5hkscs', 'cp037', 'cp273', 'cp424', 'cp437', 'cp500', 'cp720', 'cp737'
                 , 'cp775', 'cp850', 'cp852', 'cp855', 'cp856', 'cp857', 'cp858', 'cp860', 'cp861', 'cp862'
                 , 'cp863', 'cp864', 'cp865', 'cp866', 'cp869', 'cp874', 'cp875', 'cp932', 'cp949', 'cp950'
                 , 'cp1006', 'cp1026', 'cp1125', 'cp1140', 'cp1250', 'cp1251', 'cp1252', 'cp1253', 'cp1254'
                 , 'cp1255', 'cp1256', 'cp1257', 'cp1258', 'euc_jp', 'euc_jis_2004', 'euc_jisx0213', 'euc_kr'
                 , 'gb2312', 'gbk', 'gb18030', 'hz', 'iso2022_jp', 'iso2022_jp_1', 'iso2022_jp_2'
                 , 'iso2022_jp_2004', 'iso2022_jp_3', 'iso2022_jp_ext', 'iso2022_kr', 'latin_1', 'iso8859_2'
                 , 'iso8859_3', 'iso8859_4', 'iso8859_5', 'iso8859_6', 'iso8859_7', 'iso8859_8', 'iso8859_9'
                 , 'iso8859_10', 'iso8859_11', 'iso8859_13', 'iso8859_14', 'iso8859_15', 'iso8859_16', 'johab'
                 , 'koi8_r', 'koi8_t', 'koi8_u', 'kz1048', 'mac_cyrillic', 'mac_greek', 'mac_iceland', 'mac_latin2'
                 , 'mac_roman', 'mac_turkish', 'ptcp154', 'shift_jis', 'shift_jis_2004', 'shift_jisx0213', 'utf_32'
                 , 'utf_32_be', 'utf_32_le', 'utf_16', 'utf_16_be', 'utf_16_le', 'utf_7', 'utf_8', 'utf_8_sig']
for encoding in encodings:
    worked = True
    try:
        df = pd.read_csv(file, encoding=encoding, nrows=5)
    except Exception as e:
        worked = False
    if worked:
        print(encoding)
        print(df.head())
        break

ascii
  Div      Date       HomeTeam        AwayTeam  FTHG  FTAG FTR  Unnamed: 7  \
0  D1  19/08/94     Leverkusen  Kaiserslautern     0     1   A         NaN   
1  D1  19/08/94     Schalke 04      M'gladbach     1     1   D         NaN   
2  D1  19/08/94      Stuttgart         Hamburg     2     1   H         NaN   
3  D1  20/08/94  Bayern Munich          Bochum     3     1   H         NaN   
4  D1  20/08/94       Dortmund     Munich 1860     4     0   H         NaN   

   Unnamed: 8  Unnamed: 9  ...  Unnamed: 18  Unnamed: 19  Unnamed: 20  \
0         NaN         NaN  ...          NaN          NaN          NaN   
1         NaN         NaN  ...          NaN          NaN          NaN   
2         NaN         NaN  ...          NaN          NaN          NaN   
3         NaN         NaN  ...          NaN          NaN          NaN   
4         NaN         NaN  ...          NaN          NaN          NaN   

   Unnamed: 21  Unnamed: 22  Unnamed: 23  Unnamed: 24  Unnamed: 25  \
0          NaN  

In [38]:
dfs =  []
for file in os.listdir('.'):
    if file.startswith('season') or file.startswith('D1'):
        try:
            dfs.append(pd.read_csv(file, parse_dates=['Date'], dayfirst=True, usecols=['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
           'HTAG', 'HTR'], encoding='latin_1'))
        except ValueError:
            dfs.append(pd.read_csv(file, parse_dates=['Date'], dayfirst=True, usecols=['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'], encoding='latin_1'))

In [42]:
for df in dfs:
    df['season'] = f'{df.Date.min().year}-{df.Date.max().year}'

In [43]:
df = pd.concat(dfs)
df.sample(5)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,season
86,D1,2015-10-24,Leverkusen,Stuttgart,4.0,3.0,H,0.0,0.0,D,2015-2016
51,D1,2021-09-25,M'gladbach,Dortmund,1.0,0.0,H,1.0,0.0,H,2021-2022
66,D1,2004-10-16,Hertha,Leverkusen,3.0,1.0,H,1.0,0.0,H,2004-2005
45,D1,2007-09-21,Bochum,Ein Frankfurt,0.0,0.0,D,0.0,0.0,D,2007-2008
532,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999-2000


In [44]:
df.dropna(subset='HomeTeam', inplace=True)

In [46]:
df.to_csv('Bundesliga_Results.csv')

In [2]:
df = pd.read_csv('Bundesliga_Results.csv', parse_dates=['Date'])

In [45]:
# some basic cleaning and rewriting
df.sort_values(by='Date', inplace=True)
df.reset_index(inplace=True)
# clean the team names
df.HomeTeam = df.HomeTeam.apply(lambda x: x.lower().replace(' ', ''))
df.AwayTeam = df.AwayTeam.apply(lambda x: x.lower().replace(' ', ''))

# deduct winner and loser from the scores
# add draws to the winner, doesn't really matter
df['home_win'] = np.where(df.FTHG >= df.FTAG, 1, 0)
df['draw'] = np.where(df.FTHG==df.FTAG, 1, 0)
df['away_win'] = np.where(df.FTHG < df.FTAG, 1, 0)

df['winner'] = (df.HomeTeam * (df.home_win) + df.AwayTeam * df.away_win)
df['loser'] = (df.HomeTeam * (1 - df.home_win) + df.AwayTeam * (1 - df.away_win))

print('Dataset contains {} matches from {} to {}. Home won {} ({}%). There were {} draws ({}%).'.format(len(df), df.Date.min().date(), df.Date.max().date(), df.home_win.sum(), int(100*df.home_win.sum()/len(df)), df.draw.sum(), int(100*df.draw.sum()/len(df)) ))

Dataset contains 8874 matches from 1993-08-07 to 2022-05-14. Home won 6379 (71%). There were 2259 draws (25%).


In [4]:
df.tail()

,index,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Season,home_win,draw,away_win,winner,loser
10093,10018,10018,D1,2022-12-03,unionberlin,stuttgart,1,1,D,1.0,0.0,H,2021-22,1,1,0,unionberlin,stuttgart
10094,10017,10017,D1,2022-12-03,hoffenheim,bayernmunich,1,1,D,1.0,1.0,D,2021-22,1,1,0,hoffenheim,bayernmunich
10095,10016,10016,D1,2022-12-03,freiburg,wolfsburg,3,2,H,2.0,0.0,H,2021-22,1,0,0,freiburg,wolfsburg
10096,8792,8792,D1,2022-12-03,freiburg,wolfsburg,3,2,H,2.0,0.0,H,2021-22,1,0,0,freiburg,wolfsburg
10097,8795,8795,D1,2022-12-03,m'gladbach,hertha,2,0,H,1.0,0.0,H,2021-22,1,0,0,m'gladbach,hertha


In [10]:
df['month'] = df.Date.dt.month
df.tail()

,index,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Season,home_win,draw,away_win,winner,loser,month
10093,10018,10018,D1,2022-12-03,unionberlin,stuttgart,1,1,D,1.0,0.0,H,2021-22,1,1,0,unionberlin,stuttgart,12
10094,10017,10017,D1,2022-12-03,hoffenheim,bayernmunich,1,1,D,1.0,1.0,D,2021-22,1,1,0,hoffenheim,bayernmunich,12
10095,10016,10016,D1,2022-12-03,freiburg,wolfsburg,3,2,H,2.0,0.0,H,2021-22,1,0,0,freiburg,wolfsburg,12
10096,8792,8792,D1,2022-12-03,freiburg,wolfsburg,3,2,H,2.0,0.0,H,2021-22,1,0,0,freiburg,wolfsburg,12
10097,8795,8795,D1,2022-12-03,m'gladbach,hertha,2,0,H,1.0,0.0,H,2021-22,1,0,0,m'gladbach,hertha,12


In [ ]:
df.head()

In [ ]:
df.to_csv('data_preprocessed.csv')